In [411]:
import re
import pandas as pd
import folium
import osmnx as ox

## Load Mykolaiv schools and place them on the map  

In [412]:
place = 'Mykolaiv city, Ukraine'

In [413]:
tags = {"amenity": "school"}
schools = ox.features_from_place(place, tags)

In [414]:
# unique schools
schools['name'].unique()

array(['Легенда моделей', 'Курси англійської мови "Addrian"',
       'Обласна дитячо-юнацька школа олімпійського резерву з вітрильного спорту',
       'Перший центр іноземних мов', 'Іноземні мови',
       'Клуб іноземних мов', 'Школа юних моряків',
       'Дитяча школа мистецтв №3', 'Школа # 57', 'Ліцей №3',
       'Гімназія №64', 'Школа №39', 'Гімназія №32', 'Ліцей №41',
       'Школа №20', 'Школа №46', 'Академія дитячої творчості',
       'Ліцей №51', nan, 'Ліцей №22', 'Школа №26', 'Школа №10',
       'Школа-інтернат - центр загальної і профільної освіти',
       'Школа №50', 'Ліцей №19', 'Школа №45', 'Школа №57',
       'Ліцей ім. О. Ольжича', 'Школа №54',
       'Миколаївський класичний ліцей', 'Школа-інтернат №4',
       'Гімназія №47', 'Школа №11', 'Школа №7', 'Школа №48', 'Ліцей №58',
       'Школа №18', 'Школа «Гіпаніс-ЕОС»', 'Ліцей №38',
       'Муніціпальний колегіум, молодший корпус', 'Школа «Ор Менахем»',
       'Гімназія №6', 'Школа №59', 'Гімназія №44', 'Школа №17',
     

In [415]:
def add_school_columns(df, column_name, reg_map):
    """
    Enrich data frame with school columns (type and number)
 
    Args:
        df (Dataframe): source dataframe
        column_name (string): column with detailed school name
        reg_map (dict): rules for determining the school type and school number
 
    Returns:
        None
    """
    school_types = []
    school_nums = []
    
    for column, row in df.iterrows():
        school_type = None
        school_num = None
        for s_type, reg in reg_map.items():
            f = re.findall(reg, str(row[column_name]))
            if not f:
                continue
            school_type = s_type
            school_num = f[0]
            break
        school_types.append(school_type)
        school_nums.append(school_num)

    df['school_type'] = school_types
    df['school_num'] = school_nums
    df.dropna(subset=['school_type'], inplace=True)
    df['school_type'] = df['school_type'].astype('category')
    df['school_num'] = df['school_num'].astype('int')

## Add school type and school number to the dataframe

In [416]:
regexps = {'lyceum': r'Ліцей №(\d+)', 'gymnasium': r'Гімназія №(\d+)', 'school': r'Школа (?:# |№)(\d+)'}
add_school_columns(schools, 'name', regexps)
schools.loc[:, ['geometry', 'name', 'school_type', 'school_num']].head()

geometry  \
element_type osmid                                                           
node         9048530817                          POINT (31.97455 46.95815)   
way          106701569   POLYGON ((32.01679 46.96196, 32.01695 46.96195...   
             110362140   POLYGON ((32.00426 47.01483, 32.00388 47.01565...   
             110786509   POLYGON ((31.96877 46.97654, 31.96875 46.97716...   
             110860147   POLYGON ((31.99238 47.00302, 31.99401 47.00307...   

                                 name school_type  school_num  
element_type osmid                                             
node         9048530817    Школа # 57      school          57  
way          106701569       Ліцей №3      lyceum           3  
             110362140   Гімназія №64   gymnasium          64  
             110786509      Школа №39      school          39  
             110860147   Гімназія №32   gymnasium          32

### Load and prepare the rating of Nikolaev schools for 2023.

In [417]:
# From https://ru.osvita.ua/school/rating/89935/
rates = pd.read_csv("mykolaiv_school_rates_2023.csv")
rates.head()

,Назва навчального закладу,Місце,TOP,Рейт. бал,Бал НМТ,Учнів / тестів,Склав (%)
0,Миколаївський ліцей №2 Миколаївської міськради...,1,233,136.4,159.5,52/153,100
1,Миколаївський ліцей імені професора М.Александ...,2,248,136.1,158.8,69/201,100
2,Миколаївський ліцей №38 імені Володимира Дмитр...,3,266,135.7,158.4,67/195,100
3,Миколаївський ліцей імені Миколи Аркаса Микола...,4,293,135.2,158.6,26/78,100
4,Миколаївська загальноосвітня санаторна школа-і...,5,306,134.9,158.4,19/57,100


In [418]:
regexps = {'lyceum': r'ліцей №(\d+)', 'school': r'ЗОШ I-III ст. №(\d+)'}
add_school_columns(rates, 'Назва навчального закладу', regexps)
rates.head()

,Назва навчального закладу,Місце,TOP,Рейт. бал,Бал НМТ,Учнів / тестів,Склав (%),school_type,school_num
0,Миколаївський ліцей №2 Миколаївської міськради...,1,233,136.4,159.5,52/153,100,lyceum,2
2,Миколаївський ліцей №38 імені Володимира Дмитр...,3,266,135.7,158.4,67/195,100,lyceum,38
5,Миколаївський ліцей №58 Миколаївської міськрад...,6,378,134.0,156.7,57/162,99,lyceum,58
6,Миколаївський ліцей №41 Миколаївської міськрад...,7,762,130.3,152.3,57/168,100,lyceum,41
7,Миколаївський ліцей №22 Миколаївської міськрад...,8,812,130.0,151.6,80/228,99,lyceum,22


### Enrich schools dataframe with rates data

In [419]:
df = pd.merge(schools, rates, 
              how='left', left_on=['school_type','school_num'], right_on = ['school_type','school_num'])
df.dropna(subset=['name'], inplace=True)
df.loc[:, ['Назва навчального закладу', 'name', 'Місце']].head()

,Назва навчального закладу,name,Місце
0,Миколаївська ЗОШ I-III ст. №57 імені Тараса Гр...,Школа # 57,31.0
1,Миколаївський ліцей №3 Миколаївської міськради...,Ліцей №3,29.0
2,NaN,Гімназія №64,NaN
3,Миколаївська ЗОШ I-III ст. №39 імені Ю.І.Макар...,Школа №39,37.0
4,NaN,Гімназія №32,NaN


### Place schools by type on the map

In [420]:
def add_school_markers(m, df, color):
    """
    Place markers on the map
 
    Args:
        m (folium.Map): map
        df (DataFrame): source dataframe
        color (string): marker color
 
    Returns:
        None
    """
    
    for i in range(df.shape[0]):
        centroid = df.iloc[i].geometry.centroid
        location = [centroid.y, centroid.x]
        name = df['name'].iloc[i]
        
        place = '' if pd.isnull(df['Місце'].iloc[i]) else str(int(df['Місце'].iloc[i]))
        rate = '' if pd.isnull(df['Рейт. бал'].iloc[i]) else str(int(df['Рейт. бал'].iloc[i]))

        popup = f'place: {place}<br> rate: {rate}'
        
        folium.vector_layers.CircleMarker(
            location,
            radius=6,
            color='yellow',
            fill=True,
            popup=popup,
            fill_color=color,
            fill_opacity=0.6,
            tooltip=name,
        ).add_to(m)

In [421]:
m = folium.Map(location=ox.geocode(place), zoom_start=12)
add_school_markers(m, df[df['school_type'] == 'lyceum'], 'blue')
add_school_markers(m, df[df['school_type'] == 'gymnasium'], 'green')
add_school_markers(m, df[df['school_type'] == 'school'], 'red')

In [399]:
m